In [1]:
%pip install scikit-learn numpy matplotlib networkx pandas scipy torch statsmodels


Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
print(torch.version.cuda)

11.7


In [3]:
%pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# tu trzeba wpisać odpowiednią wersję CUDA, patrz: https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html

Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu117.html
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.


In [5]:
from torch_geometric.datasets import TUDataset

ROOT_DATA_PATH = "../data"

imdb_binary_dataset = TUDataset(root=ROOT_DATA_PATH, name="IMDB-BINARY")
imdb_multi_dataset = TUDataset(root=ROOT_DATA_PATH, name="IMDB-MULTI")
reddit_binary_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-BINARY")
reddit_multi_5k_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-MULTI-5K")
reddit_multi_12k_dataset = TUDataset(root=ROOT_DATA_PATH, name="REDDIT-MULTI-12K")

# do pobrania jeszcze dataset z cząsteczkami stąd: https://ogb.stanford.edu/docs/graphprop/#pyg

In [6]:
HYPERPARAMS = {
    'n_bin': 30,  # number of bins for historgram
    'norm_flag': 'no',  # normalize before calling function_basis versus normalize after
    'nonlinear_flag': 'True',  # SVM linear kernel versus nonlinear kernel
    'uniform_flag': True,  # unform versus log scale. True for imdb, False for reddit.
    'cdf_flag': True,  # cdf versus pdf. True for most dataset.
    'his_norm_flag': 'yes'
}

In [7]:
from main import evaluate

evaluate(dataset="imdb_binary", classifier='svm', baseline='ldp', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
imdb_binary
Evaluating with SVM
[0.77 0.73 0.75 0.78 0.69 0.69 0.67 0.76 0.74 0.69]
[0.72 0.77 0.69 0.73 0.74 0.64 0.79 0.69 0.76 0.78]
[0.71 0.73 0.73 0.69 0.7  0.73 0.75 0.74 0.76 0.77]
[0.71 0.72 0.74 0.71 0.8  0.71 0.67 0.69 0.76 0.71]
[0.71 0.75 0.78 0.73 0.77 0.71 0.7  0.7  0.69 0.73]
[0.77 0.74 0.68 0.77 0.73 0.72 0.81 0.72 0.65 0.64]
[0.7  0.68 0.75 0.68 0.79 0.71 0.73 0.67 0.78 0.66]
[0.73 0.71 0.73 0.81 0.68 0.76 0.7  0.73 0.64 0.72]
[0.74 0.79 0.66 0.76 0.69 0.74 0.77 0.76 0.7  0.67]
[0.68 0.66 0.8  0.7  0.78 0.75 0.75 0.7  0.72 0.76]
mean is 0.7254999999999999, std is 0.00490407993409567 
'evaluate_svm'  6.45 s


In [8]:
evaluate(dataset="imdb_binary", classifier='random_forest', baseline='ldp', hyperparams=HYPERPARAMS)

Loading graph from dataset imdb_binary
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
Evaluating with Random Forest
Accuracy: 0.665


In [9]:
evaluate(dataset="IMDB-BINARY", classifier='svm', baseline='ldp', hyperparams=HYPERPARAMS)

READ GRAPH
GRAPH READED
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
IMDB-BINARY
No best SVM params found for IMDB-BINARY, using defaults.
Evaluating with SVM
[0.77 0.73 0.75 0.78 0.69 0.69 0.67 0.76 0.74 0.69]
[0.72 0.77 0.69 0.73 0.74 0.64 0.79 0.69 0.76 0.78]
[0.71 0.73 0.73 0.69 0.7  0.73 0.75 0.74 0.76 0.77]
[0.71 0.72 0.74 0.71 0.8  0.71 0.67 0.69 0.76 0.71]
[0.71 0.75 0.78 0.73 0.77 0.71 0.7  0.7  0.69 0.73]
[0.77 0.74 0.68 0.77 0.73 0.72 0.81 0.72 0.65 0.64]
[0.7  0.68 0.75 0.68 0.79 0.71 0.73 0.67 0.78 0.66]
[0.73 0.71 0.73 0.81 0.68 0.76 0.7  0.73 0.64 0.72]
[0.74 0.79 0.66 0.76 0.69 0.74 0.77 0.76 0.7  0.67]
[0.68 0.66 0.8  0.7  0.78 0.75 0.75 0.7  0.72 0.76]
mean is 0.7254999999999999, std is 0.00490407993409567 
'evaluate_svm'  5.81 s


In [10]:
evaluate(dataset="IMDB-BINARY", classifier='random_forest', baseline='ldp', hyperparams=HYPERPARAMS)

READ GRAPH
GRAPH READED
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
Number of bins are 30
the shape after removing zero columns is  (1000, 145)
Evaluating with Random Forest
Accuracy: 0.69
